In [1]:
def count_parameters(model): 
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

### Reversible

In [2]:
import expconfigs.multi_atlas_revunet_01_v2 as cfg_rev
excfg_rev = cfg_rev.ExpConfig()
nr = (count_parameters(excfg_rev.net))
print(excfg_rev.experiment_name)

atlas_revunet_3D_5_v2


### Classic

In [3]:
import expconfigs.multi_atlas_unet_res01_v2 as cfg_cla
excfg_cla = cfg_cla.ExpConfig()
nc = (count_parameters(excfg_cla.net))

/home/myr/these/segmentation3D/models/networks_other.py:42: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight.data, a=0, mode='fan_in')


### Compare

In [6]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 114557582
cla : 94122318
nc/nr : 0.8216157879449655
nr/nc : 1.2171139049082917
min : 0.8216157879449655


In [4]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 76837774
cla : 94122318
nc/nr : 1.2249485259684905
nr/nc : 0.8163608337822704
min : 0.8163608337822704


In [4]:
print("rev :", nr)
print("cla :", nc)
print("nc/nr :", nc/nr)
print("nr/nc :", nr/nc)
print('min :', min(nc/nr, nr/nc))

rev : 39117966
cla : 94122318
nc/nr : 2.406114827135951
nr/nc : 0.41560776265624905
min : 0.41560776265624905
